In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [10]:
prices_path = "../data/island-data-bottle-round-1/prices_round_1_day_"
trades_path = "../data/island-data-bottle-round-1/trades_round_1_day_"

price_files = [prices_path + str(i) + ".csv" for i in range(-2, 1, 1)]
trade_files = [trades_path + str(i) + "_nn.csv" for i in range(-2, 1, 1)]

price_files,  trade_files

(['../data/island-data-bottle-round-1/prices_round_1_day_-2.csv',
  '../data/island-data-bottle-round-1/prices_round_1_day_-1.csv',
  '../data/island-data-bottle-round-1/prices_round_1_day_0.csv'],
 ['../data/island-data-bottle-round-1/trades_round_1_day_-2_nn.csv',
  '../data/island-data-bottle-round-1/trades_round_1_day_-1_nn.csv',
  '../data/island-data-bottle-round-1/trades_round_1_day_0_nn.csv'])

In [11]:
# Import and clean data
prices_df = pd.concat((pd.read_csv(filename, sep=";", index_col=["day", "timestamp", "product"]) for filename in price_files)).drop(["profit_and_loss"], axis="columns")
trades_df = pd.concat((pd.read_csv(filename, sep=";", usecols=lambda x : x not in ["buyer", "seller", "currency"]).assign(day=day-2) for day, filename in enumerate(trade_files))).set_index(["day", "timestamp", "symbol"])

In [12]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 60000 entries, (-2, 0, 'BANANAS') to (0, 999900, 'BANANAS')
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   bid_price_1   60000 non-null  int64  
 1   bid_volume_1  60000 non-null  int64  
 2   bid_price_2   33401 non-null  float64
 3   bid_volume_2  33401 non-null  float64
 4   bid_price_3   6304 non-null   float64
 5   bid_volume_3  6304 non-null   float64
 6   ask_price_1   60000 non-null  int64  
 7   ask_volume_1  60000 non-null  int64  
 8   ask_price_2   32930 non-null  float64
 9   ask_volume_2  32930 non-null  float64
 10  ask_price_3   6619 non-null   float64
 11  ask_volume_3  6619 non-null   float64
 12  mid_price     60000 non-null  float64
dtypes: float64(9), int64(4)
memory usage: 6.5+ MB


In [13]:
prices_df.sample(10)

bid_price_1  bid_volume_1  bid_price_2  bid_volume_2  \
day timestamp product                                                         
 0  325000    PEARLS          9996             2       9995.0          21.0   
    156800    BANANAS         4900             1       4899.0          25.0   
    313400    PEARLS         10002             2       9996.0           1.0   
-1  173000    BANANAS         4927            31          NaN           NaN   
    185200    PEARLS         10002             1       9995.0          26.0   
    491600    BANANAS         4911            27          NaN           NaN   
-2  944500    PEARLS          9996             1       9995.0          30.0   
-1  675800    BANANAS         4912            27          NaN           NaN   
    372100    BANANAS         4933            32          NaN           NaN   
 0  292000    BANANAS         4890            26          NaN           NaN   

                       bid_price_3  bid_volume_3  ask_price_1  ask_volume_1  \
day timestamp product                                                         
 0  325000    PEARLS           NaN           NaN        10004             2   
    156800    BANANAS          NaN           NaN         4906            25   
    313400    PEARLS        9995.0          27.0        10004             1   
-1  173000    BANANAS          NaN           NaN         4934            31   
    185200    PEARLS           NaN           NaN        10005            26   
    491600    BANANAS          NaN           NaN         4918            28   
-2  944500    PEARLS           NaN           NaN        10002             4   
-1  675800    BANANAS          NaN           NaN         4914             4   
    372100    BANANAS          NaN           NaN         4939            32   
 0  292000    BANANAS          NaN           NaN         4896             1   

                       ask_price_2  ask_volume_2  ask_price_3  ask_volume_3  \
day timestamp product                                                         
 0  325000    PEARLS       10005.0          21.0          NaN           NaN   
    156800    BANANAS          NaN           NaN          NaN           NaN   
    313400    PEARLS       10005.0          27.0          NaN           NaN   
-1  173000    BANANAS          NaN           NaN          NaN           NaN   
    185200    PEARLS           NaN           NaN          NaN           NaN   
    491600    BANANAS          NaN           NaN          NaN           NaN   
-2  944500    PEARLS       10004.0           1.0      10005.0          30.0   
-1  675800    BANANAS       4919.0          27.0          NaN           NaN   
    372100    BANANAS          NaN           NaN          NaN           NaN   
 0  292000    BANANAS       4897.0          25.0          NaN           NaN   

                       mid_price  
day timestamp product             
 0  325000    PEARLS     10000.0  
    156800    BANANAS     4903.0  
    313400    PEARLS     10003.0  
-1  173000    BANANAS     4930.5  
    185200    PEARLS     10003.5  
    491600    BANANAS     4914.5  
-2  944500    PEARLS      9999.0  
-1  675800    BANANAS     4913.0  
    372100    BANANAS     4936.0  
 0  292000    BANANAS     4893.0

In [14]:
prices_agg_df = prices_df[["mid_price"]]
prices_agg_df.describe()

,mid_price
count,60000.000000
mean,7464.937350
std,2535.227736
min,4869.500000
25%,4919.500000
50%,7501.500000
75%,10000.000000
max,10003.500000


In [15]:
prices_agg_df.groupby("product").describe()

mid_price                                                            \
            count        mean        std     min      25%      50%      75%   
product                                                                       
BANANAS   30000.0  4929.87815  38.634426  4869.5   4894.0   4919.5   4969.5   
PEARLS    30000.0  9999.99655   1.496592  9996.5  10000.0  10000.0  10000.0   

                  
             max  
product           
BANANAS   5006.5  
PEARLS   10003.5

In [16]:
prices_agg_df = prices_agg_df.groupby(["product", "day"])

In [17]:
prices_agg_df.describe()

mid_price                                                    \
                count         mean        std     min      25%      50%   
product day                                                               
BANANAS -2    10000.0   4977.97640  12.163889  4942.5   4969.5   4980.0   
        -1    10000.0   4922.15040  16.393819  4886.5   4911.0   4919.5   
         0    10000.0   4889.50765   7.622357  4869.5   4884.0   4890.0   
PEARLS  -2    10000.0  10000.00350   1.513842  9996.5  10000.0  10000.0   
        -1    10000.0   9999.98640   1.479606  9996.5  10000.0  10000.0   
         0    10000.0   9999.99975   1.496228  9996.5  10000.0  10000.0   

                               
                 75%      max  
product day                    
BANANAS -2    4987.0   5006.5  
        -1    4934.5   4957.0  
         0    4894.5   4911.0  
PEARLS  -2   10000.0  10003.5  
        -1   10000.0  10003.5  
         0   10000.0  10003.5

In [18]:
trades_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15786 entries, (-2, 0, 'BANANAS') to (0, 999500, 'PEARLS')
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     15786 non-null  float64
 1   quantity  15786 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 624.1+ KB


In [19]:
trades_df.sample(10)

price  quantity
day timestamp symbol                    
-1  642700    PEARLS    9998.0         1
    76400     BANANAS   4942.0         8
 0  747700    BANANAS   4883.0         1
    866700    BANANAS   4881.0        17
    276000    BANANAS   4891.0         4
-2  266500    BANANAS   4995.0         1
-1  92700     BANANAS   4933.0         1
    419500    BANANAS   4922.0         2
 0  289900    PEARLS   10005.0         2
    84500     BANANAS   4891.0         1

In [20]:
trades_agg_df = trades_df.groupby(["symbol", "day"])

In [21]:
trades_agg_df.describe()

price                                                    \
              count          mean        std     min     25%      50%   
symbol  day                                                             
BANANAS -2   2896.0   4975.893992  93.280105     0.0  4969.0   4979.0   
        -1   2892.0   4921.540456  16.646177  4886.0  4910.0   4919.0   
         0   2763.0   4889.246833   7.967840  4869.0  4883.0   4889.0   
PEARLS  -2   2444.0  10000.038462   3.139960  9995.0  9998.0  10000.0   
        -1   2387.0   9999.897361   3.183008  9995.0  9998.0   9998.0   
         0   2404.0   9999.967970   3.132165  9995.0  9998.0   9998.0   

                              quantity                                     \
                 75%      max    count      mean       std  min  25%  50%   
symbol  day                                                                 
BANANAS -2    4987.0   5006.0   2896.0  2.718232  3.572302  0.0  1.0  1.0   
        -1    4934.0   4958.0   2892.0  2.800830  3.702593  0.0  1.0  1.0   
         0    4895.0   4911.0   2763.0  2.740861  3.609488  0.0  1.0  1.0   
PEARLS  -2   10002.0  10005.0   2444.0  2.118658  1.985167  0.0  1.0  1.0   
        -1   10002.0  10005.0   2387.0  2.154587  2.094174  0.0  1.0  1.0   
         0   10002.0  10005.0   2404.0  2.040765  1.925582  0.0  1.0  1.0   

                        
             75%   max  
symbol  day             
BANANAS -2   2.0  20.0  
        -1   2.0  20.0  
         0   2.0  20.0  
PEARLS  -2   2.0  12.0  
        -1   2.0  12.0  
         0   2.0  12.0

In [22]:
prices_agg_df.describe()

mid_price                                                    \
                count         mean        std     min      25%      50%   
product day                                                               
BANANAS -2    10000.0   4977.97640  12.163889  4942.5   4969.5   4980.0   
        -1    10000.0   4922.15040  16.393819  4886.5   4911.0   4919.5   
         0    10000.0   4889.50765   7.622357  4869.5   4884.0   4890.0   
PEARLS  -2    10000.0  10000.00350   1.513842  9996.5  10000.0  10000.0   
        -1    10000.0   9999.98640   1.479606  9996.5  10000.0  10000.0   
         0    10000.0   9999.99975   1.496228  9996.5  10000.0  10000.0   

                               
                 75%      max  
product day                    
BANANAS -2    4987.0   5006.5  
        -1    4934.5   4957.0  
         0    4894.5   4911.0  
PEARLS  -2   10000.0  10003.5  
        -1   10000.0  10003.5  
         0   10000.0  10003.5

In [23]:
pearls_trades_df = trades_df.xs(key="PEARLS", axis=0, level="symbol")
pearls_trades_df


price  quantity
day timestamp                   
-2  0           9996.0         1
    0           9996.0         1
    1500       10004.0         1
    2500        9998.0         1
    3200        9996.0         1
...                ...       ...
 0  997600     10002.0         4
    997700     10002.0         1
    998400     10002.0         4
    998800     10002.0         1
    999500      9995.0         2

[7235 rows x 2 columns]

In [36]:
cont_prices_df = prices_df.reset_index()
banana_prices_df = cont_prices_df[cont_prices_df["product"] == "BANANAS"]
pearl_prices_df = cont_prices_df[cont_prices_df["product"] == "PEARLS"]

In [39]:
banana_prices_df.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price
0,-2,0,BANANAS,4997,26,NaN,NaN,NaN,NaN,5003,26,NaN,NaN,NaN,NaN,5000.0
2,-2,100,BANANAS,4997,20,NaN,NaN,NaN,NaN,4998,1,5004.0,20.0,NaN,NaN,4997.5
5,-2,200,BANANAS,4997,31,NaN,NaN,NaN,NaN,5004,31,NaN,NaN,NaN,NaN,5000.5
7,-2,300,BANANAS,4997,27,NaN,NaN,NaN,NaN,5004,27,NaN,NaN,NaN,NaN,5000.5
9,-2,400,BANANAS,4997,20,NaN,NaN,NaN,NaN,5002,9,5004.0,20.0,NaN,NaN,4999.5


In [40]:
pearl_prices_df.head()

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price
1,-2,0,PEARLS,9996,1,9995.0,25.0,NaN,NaN,10004,1,10005.0,25.0,NaN,NaN,10000.0
3,-2,100,PEARLS,10000,5,9995.0,20.0,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10002.5
4,-2,200,PEARLS,9996,1,9995.0,30.0,NaN,NaN,10004,1,10005.0,30.0,NaN,NaN,10000.0
6,-2,300,PEARLS,9996,1,9995.0,26.0,NaN,NaN,10004,1,10005.0,26.0,NaN,NaN,10000.0
8,-2,400,PEARLS,9995,20,NaN,NaN,NaN,NaN,9998,3,10002.0,9.0,10005.0,20.0,9996.5


In [47]:
banana_prices_df["avg_price"] = banana_prices_df.apply(lambda row: (row.ask_price_1 + row.bid_price_1) / 2, axis=1)
banana_prices_df.sample(100)

/var/folders/j4/5rkww2t94332tqwmg7r7y2k40000gn/T/ipykernel_23665/2117430832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  banana_prices_df["avg_price"] = banana_prices_df.apply(lambda row: (row.ask_price_1 + row.bid_price_1) / 2, axis=1)


,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,avg_price
52135,0,606700,BANANAS,4894,27,NaN,NaN,NaN,NaN,4899,7,4901.0,27.0,NaN,NaN,4896.5,4896.5
47218,0,360900,BANANAS,4888,6,4885.0,31.0,NaN,NaN,4891,1,4892.0,30.0,NaN,NaN,4889.5,4889.5
3262,-2,163100,BANANAS,4973,23,NaN,NaN,NaN,NaN,4978,6,4980.0,23.0,NaN,NaN,4975.5,4975.5
17451,-2,872500,BANANAS,4967,23,NaN,NaN,NaN,NaN,4974,23,NaN,NaN,NaN,NaN,4970.5,4970.5
30438,-1,521900,BANANAS,4916,2,4915.0,21.0,NaN,NaN,4922,23,NaN,NaN,NaN,NaN,4919.0,4919.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43593,0,179600,BANANAS,4902,1,4897.0,2.0,4896.0,27.0,4903,29,NaN,NaN,NaN,NaN,4902.5,4902.5
22368,-1,118400,BANANAS,4925,30,NaN,NaN,NaN,NaN,4931,30,NaN,NaN,NaN,NaN,4928.0,4928.0
35158,-1,757900,BANANAS,4904,2,4903.0,21.0,NaN,NaN,4910,23,NaN,NaN,NaN,NaN,4907.0,4907.0
1871,-2,93500,BANANAS,4975,28,NaN,NaN,NaN,NaN,4982,28,NaN,NaN,NaN,NaN,4978.5,4978.5
